# Hapag-Lloyd

In [11]:
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
import json
import time

In [12]:
container = {
    "year": "2019",
    "manifest": "450",
    "detail": "874",
    "container": "HLXU5183586",
    "carrier": "Hapag-Lloyd"
}

In [15]:
with open("../../config/" + container["carrier"] + ".json") as file:
    config = json.load(file)

In [16]:
config

{'name': 'Hapag-Lloyd',
 'general': {'url': 'https://www.hapag-lloyd.com/en/online-business/tracing/tracing-by-container.html?container={container}',
  'date_format': '%Y-%m-%d',
  'time_format': '%H:%M'},
 'input': [{'type': 'class',
   'value': 'error',
   'command': {'type': 'assert', 'value': False}},
  {'type': 'id',
   'value': 'hal-cookieconsent-button',
   'command': {'type': 'click', 'value': False, 'required': False}}],
 'output': [{'type': 'class',
   'value': 'inputNonEdit',
   'commands': [{'index': 0,
     'type': 'save',
     'value': 'container_type',
     'required': False},
    {'index': 1,
     'type': 'save',
     'value': 'container_description',
     'required': True},
    {'index': 2,
     'type': 'split',
     'value': ' X ',
     'commands': [{'index': 0,
       'type': 'save',
       'value': 'container_length',
       'required': False},
      {'index': 2,
       'type': 'save',
       'value': 'container_height',
       'required': False},
      {'index': 1,

## Input commands

In [9]:
driver = webdriver.Firefox(executable_path = "../../driver/geckodriver")
link = "https://www.hapag-lloyd.com/en/online-business/tracing/tracing-by-container.html?container={container}"
container = {
    "container": "HLXU5183586"
}
driver.get(link.format(**container))

initial_time = time.time()
while True:
    try:
        do_input_commands()
button = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.ID, "hal-cookieconsent-button")))
button.click()

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="add64742-6c12-44b9-b7a0-f43bb8f30ad2", element="08cf1b24-1394-4863-9816-8185825dc6a6")>


In [10]:
driver.close()

In [11]:
elements = driver.find_elements(By.CLASS_NAME, "inputNonEdit")

# Container type
tracking["general"]["container_type"] = elements[0].text

# Container description
tracking["general"]["container_description"] = elements[1].text

# Container dimensions
text_group = elements[2].text.split(" X ")
tracking["general"]["container_length"] = text_group[0]
tracking["general"]["container_height"] = text_group[2]
tracking["general"]["container_width"]  = text_group[1]

# Container tare
tracking["container"]["tare"] = elements[3].text

# Container maximum payload
tracking["container"]["max_payload"] = elements[4].text

In [15]:
elements = driver.find_elements(By.CSS_SELECTOR, '.hal-table tbody tr')
tracking["movements"] = []

for element in elements:
    # Get items and create new movement
    items = element.find_elements(By.TAG_NAME, 'td')
    movement = {
        "transport": {}
    }
    
    # Container status in movement
    movement["status"] = items[0].text
    
    # Container location in movement
    movement["location"] = items[1].text
    
    # Container movement date
    movement["date"] = items[2].text
    
    # Container movement time
    movement["time"] = items[3].text
    
    # Container movement transport ship
    movement["transport_ship"] = items[4].text
    
    # Container movement transport voyage number
    try:
        text = items[5].text
        if (len(text) > 0):
            movement["transport_voyage"] = text
    except IndexError:
        pass
    
    # Add movement to list
    tracking["movements"].append(movement)


In [16]:
tracking

{'container': {'description': 'REEFER CONTAINER',
  'dimensions': {'height': '9\'6"', 'length': "40'", 'width': "8'"},
  'max_payload': '29360',
  'tare': '4640',
  'type': '45RT'},
 'general': {},
 'last_route': {},
 'movements': [{'date': '2019-01-26',
   'location': 'CALLAO',
   'status': 'Gate out empty',
   'time': '09:59',
   'transport_ship': 'Truck'},
  {'date': '2019-01-27',
   'location': 'CALLAO',
   'status': 'Arrival in',
   'time': '06:01',
   'transport_ship': 'Truck'},
  {'date': '2019-01-29',
   'location': 'CALLAO',
   'status': 'Loaded',
   'time': '07:51',
   'transport_ship': 'CAP SAN TAINARO',
   'transport_voyage': '8151N'},
  {'date': '2019-01-29',
   'location': 'CALLAO',
   'status': 'Vessel departed',
   'time': '23:42',
   'transport_ship': 'CAP SAN TAINARO',
   'transport_voyage': '8151N'},
  {'date': '2019-02-19',
   'location': 'ROTTERDAM',
   'status': 'Vessel arrived',
   'time': '16:42',
   'transport_ship': 'CAP SAN TAINARO',
   'transport_voyage': '8

In [17]:
driver.close()

In [18]:
import json
with open('hapag-lloyd/' + container + '.json', 'w') as file:
    json.dump(tracking, file)